In [4]:
from trees import *
from black_scholes import *

Set Inputs

In [5]:
inputs = set_inputs(asset=12, volatility=0.3, interest_rate=0.06, dividend=0.04, strike=10, time_to_maturity=1, time_periods=4, u=None, d=None, barrier=None)
inputs

{'asset': 12,
 'sigma': 0.3,
 'r': 0.06,
 'q': 0.04,
 'K': 10,
 'T': 1,
 'periods': 4,
 'step': 0.25,
 'u': None,
 'd': None,
 'barrier': None}

In [18]:
d = get_d(interest_rate=inputs["r"], dividend=inputs["q"], volatility=inputs["sigma"], step=inputs["step"], d=inputs["d"])
u = get_u(d=d, u=inputs["u"])
u, d

(1.1631873818945189, 0.8597067124054161)

In [19]:
stock_tree = get_stock_tree(asset=inputs["asset"], u=u, d=d, time_periods=inputs["periods"])
print_tree(stock_tree, inputs["periods"])

[12.]
[10.31648055 13.95824858]
[ 8.86914758 12.         12.         16.23605862]
[ 7.6248657  10.31648055 10.31648055 13.95824858 10.31648055 13.95824858
 13.95824858 18.88557852]
[ 6.55514823  8.86914758  8.86914758 12.          8.86914758 12.
 12.         16.23605862  8.86914758 12.         12.         16.23605862
 12.         16.23605862 16.23605862 21.96746664]


In [20]:
p = get_p(interest_rate=inputs["r"], dividend=inputs["q"], time_periods=inputs["periods"], step=inputs["step"], tree=stock_tree)
p

array([0.        , 0.47879757, 0.47879757, 0.47879757, 0.47879757,
       0.47879757, 0.47879757, 0.47879757, 0.47879757, 0.47879757,
       0.47879757, 0.47879757, 0.47879757, 0.47879757, 0.47879757,
       0.47879757])

In [21]:
maturity_func_put = put_at_maturity
maturity_func_call = call_at_maturity

In [22]:
tree_func_am_put = american_put_tree
tree_func_am_call = american_call_tree
tree_func_eu_put = european_option_tree
tree_func_eu_call = european_option_tree

In [26]:
american_put = get_asset_tree(stocks=stock_tree, strike=inputs["K"], interest_rate=inputs["r"], time_periods=inputs["periods"], p=p, step=inputs["step"], maturity_func=maturity_func_put, tree_func=tree_func_am_put, barrier=None)
print_tree(american_put, inputs["periods"])

[0.53807875]
[0.90736837 0.15306714]
[1.49335977 0.29811921 0.29811921 0.        ]
[2.3751343  0.58062796 0.58062796 0.         0.58062796 0.
 0.         0.        ]
[3.44485177 1.13085242 1.13085242 0.         1.13085242 0.
 0.         0.         1.13085242 0.         0.         0.
 0.         0.         0.         0.        ]


In [27]:
european_put = get_asset_tree(stocks=stock_tree, strike=inputs["K"], interest_rate=inputs["r"], time_periods=inputs["periods"], p=p, step=inputs["step"], maturity_func=maturity_func_put, tree_func=tree_func_eu_put, barrier=None)
print_tree(european_put, inputs["periods"])

[0.52819618]
[0.88812072 0.15306714]
[1.45587233 0.29811921 0.29811921 0.        ]
[2.30212237 0.58062796 0.58062796 0.         0.58062796 0.
 0.         0.        ]
[3.44485177 1.13085242 1.13085242 0.         1.13085242 0.
 0.         0.         1.13085242 0.         0.         0.
 0.         0.         0.         0.        ]


In [28]:
american_put_ex = exercise_american(european_option=european_put, american_option=american_put)
american_put_ex, american_put[american_put_ex]

(array([[ 1],
        [ 3],
        [ 7],
        [15]]),
 array([[0.53807875],
        [0.90736837],
        [1.49335977],
        [2.3751343 ]]))

In [29]:
american_put_delta = get_delta_tree(stocks=stock_tree, options=american_put, time_periods=inputs["periods"])
print_tree(american_put_delta, inputs["periods"]//2)
print(get_gamma_tree(american_put_delta, inputs["periods"]))

[-0.20712501]
[-0.38176203 -0.07037655]
[-0.6667025  -0.15943573 -0.15943573  0.        ]
[0.         0.31138547 0.15943573 0.50726676 0.         0.36119634
 0.36119634 0.63880366]


In [30]:
european_put_delta = get_delta_tree(stocks=stock_tree, options=european_put, time_periods=inputs["periods"])
print_tree(european_put_delta, inputs["periods"]//2)
print(get_gamma_tree(european_put_delta, inputs["periods"]))

[-0.20183976]
[-0.36978847 -0.07037655]
[-0.6395768  -0.15943573 -0.15943573  0.        ]
[0.         0.29941192 0.15943573 0.48014107 0.         0.36119634
 0.36119634 0.63880366]


In [31]:
put_price_control_variate = control_variate(american_put[1], european_put[1], bs_put(S=inputs["asset"], K=inputs["K"], T=inputs["T"], r=inputs["r"], sigma=inputs["sigma"]))
put_price_control_variate

0.38553957263035943

In [32]:
put_delta_control_variate = control_variate(american_put_delta[1], european_put_delta[1], put_delta(S=inputs["asset"], K=inputs["K"], T=inputs["T"], r=inputs["r"], sigma=inputs["sigma"]))
put_delta_control_variate

-0.1743825630586943

In [34]:
american_call = get_asset_tree(stocks=stock_tree, strike=inputs["K"], interest_rate=inputs["r"], time_periods=inputs["periods"], p=p, step=inputs["step"], maturity_func=maturity_func_call, tree_func=tree_func_am_call, barrier=None)
print_tree(american_call, inputs["periods"])

[2.64579745]
[1.33972837 4.15105252]
[0.44494368 2.35604795 2.35604795 6.23605862]
[0.         0.94333841 0.94333841 3.96824229 0.94333841 3.96824229
 3.96824229 8.88557852]
[ 0.          0.          0.          2.          0.          2.
  2.          6.23605862  0.          2.          2.          6.23605862
  2.          6.23605862  6.23605862 11.96746664]


In [36]:
european_call = get_asset_tree(stocks=stock_tree, strike=inputs["K"], interest_rate=inputs["r"], time_periods=inputs["periods"], p=p, step=inputs["step"], maturity_func=maturity_func_call, tree_func=tree_func_eu_call, barrier=None)
print_tree(european_call, inputs["periods"])

[2.64002412]
[1.33972837 4.13881231]
[0.44494368 2.35604795 2.35604795 6.21010779]
[0.         0.94333841 0.94333841 3.96824229 0.94333841 3.96824229
 3.96824229 8.84654448]
[ 0.          0.          0.          2.          0.          2.
  2.          6.23605862  0.          2.          2.          6.23605862
  2.          6.23605862  6.23605862 11.96746664]


In [37]:
american_call_ex = exercise_american(european_option=european_call, american_option=american_call)
american_call_ex, american_call[american_call_ex]

(array([[1],
        [2],
        [4],
        [8]]),
 array([[2.64579745],
        [4.15105252],
        [6.23605862],
        [8.88557852]]))

In [38]:
american_call_delta = get_delta_tree(stocks=stock_tree, options=american_call, time_periods=inputs["periods"])
print_tree(american_call_delta, inputs["periods"])
american_call_delta, american_call_delta[american_call_ex[:-1]]

[0.77196684]
[0.6104102 0.9159483]
[0.35047303 0.8306141  0.8306141  0.99797178]
[0.         0.63880366 0.63880366 1.         0.63880366 1.
 1.         1.        ]
[]


(array([0.        , 0.77196684, 0.9159483 , 0.6104102 , 0.99797178,
        0.8306141 , 0.8306141 , 0.35047303, 1.        , 1.        ,
        1.        , 0.63880366, 1.        , 0.63880366, 0.63880366,
        0.        ]),
 array([[0.77196684],
        [0.9159483 ],
        [0.99797178]]))

In [39]:
european_call_delta = get_delta_tree(stocks=stock_tree, options=european_call, time_periods=inputs["periods"])
print_tree(european_call_delta, inputs["periods"])

[0.76860578]
[0.6104102  0.90982212]
[0.35047303 0.8306141  0.8306141  0.99004983]
[0.         0.63880366 0.63880366 1.         0.63880366 1.
 1.         1.        ]
[]


In [40]:
call_price_control_variate = control_variate(american_call[1], european_call[1], bs_call(S=inputs["asset"], K=inputs["K"], T=inputs["T"], r=inputs["r"], sigma=inputs["sigma"]))
call_price_control_variate

2.963784999033424

In [41]:
call_delta_control_variate = control_variate(american_call_delta[1], european_call_delta[1], call_delta(S=inputs["asset"], K=inputs["K"], T=inputs["T"], r=inputs["r"], sigma=inputs["sigma"]))
call_delta_control_variate

0.8342637494857015

In [42]:
def func(stock, strike, barrier):
    return np.log(stock)


In [48]:
derivative_tree = get_asset_tree(stocks=stock_tree, strike=inputs["K"], interest_rate=inputs["r"], time_periods=inputs["periods"], p=p, step=inputs["step"], maturity_func=func, tree_func=tree_func_eu_call, barrier=None)
print_tree(derivative_tree, inputs["periods"])

TypeError: func() takes 2 positional arguments but 3 were given

In [ ]:
stock_tree = [0, 100, 150, 130, 170, 100, 100, 70]
time_periods = int(np.log2(len(stock_tree)) - 1)
p = get_p(interest_rate=0.06, dividend=0, time_periods=time_periods, step=1, tree=stock_tree)
american_call = get_asset_tree(stocks=stock_tree, strike=10, interest_rate=0.06, time_periods=time_periods, p=p, step=1, maturity_func=maturity_func_call, tree_func=tree_func_am_call, barrier=None)
print_tree(american_call, 2)

[91.13079563]
[120.58235466 140.58235466]
[ 60.  90.  90. 160.]
